In [10]:
import io
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
import json
import spacy

trainingJson = json.load(open("twitter_training.json"))

#generate tokenizer for entire dataset

tokenizer = get_tokenizer('basic_english')

counter = Counter()

max_vocab = 10
vlen = 0

for user in trainingJson["data"]:
    if vlen > max_vocab:
        break
    vlen += 1
    counter.update(tokenizer(user["bio"]))
    for act in user["activity"]:
        counter.update(tokenizer(act))

vocab = Vocab(counter)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")

In [11]:
# def data_process(raw_text_iter):
#   data = [torch.tensor([vocab[token] for token in tokenizer(item)],
#                        dtype=torch.long) for item in raw_text_iter]
#   return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
#

# def read_output(output_batch):
#     for batch in output_batch:
#         for


def data_process(raw_text_iter):
    print(raw_text_iter)
    batch = []
    for item in raw_text_iter:
        data = []
        token = tokenizer(item)
        for tok in token:
            ch = [0] * len(vocab.stoi)
            ch[vocab[tok]] = 1
            data.append(ch)

    batch.append(data)

    return torch.tensor(batch, dtype=torch.float)



In [12]:
batchSize = 1


# def get_labels(raw_text_iter):
#
#     data = []
#     for item in raw_text_iter:
#
#         token = tokenizer(item)
#         for tok in token:
#             ch = [0] * len(vocab.stoi)
#             ch[vocab[tok]] = 1
#             data.append(ch)
#
#
#     batch = []
#
#     batch.append(data)
#
#     return torch.tensor(batch, dtype=torch.float)


def get_batch(i,l):
    user_bio = trainingJson["data"][i]["bio"]

    user_acts = ""
    for i in range(0,l):
        user_acts += " " +trainingJson["data"][i]["activity"][i]

    return data_process(user_acts).to(device), data_process(user_bio).to(device)

In [13]:
import math

import torch
import torch.nn as nn

ntokens = len(vocab.stoi)  # the size of vocabulary
emsize = 200  # embedding dimension
nhid = 200  # the dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # the number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # the number of heads in the multiheadattention models
dropout = 0.2  # the dropout value



In [14]:
transformer_model = nn.Transformer(d_model=ntokens, nhead=2, num_encoder_layers=12).to(device)

total_loss = 0

lr = 5.0 # learning rate
optimizer = torch.optim.SGD(transformer_model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

criterion = nn.BCEWithLogitsLoss()

for batch in range(0,10):
    optimizer.zero_grad()

    src, tgt = get_batch(batch,10)

    transformer_model.train()

    out = transformer_model(src, tgt)

    loss = criterion(out, tgt)

    loss.backward()
    torch.nn.utils.clip_grad_norm_(transformer_model.parameters(), 0.5)
    optimizer.step()

    total_loss += loss.item()
    log_interval = 200

    if batch % 1 == 0 and batch > 0:
        cur_loss = total_loss / log_interval
        print(total_loss , " total loss")
        total_loss = 0

 RT @MargBarAmerica: i love seeing dudes who look like they've never lifted a weight in their life freak out about potentially eating less r… RT @abrahams_wife: found one of my classic hits on instagram https://t.co/TNRqMKjV7N @TanyaCharbury Thank you. I followed my own pioneers. Jan Morris, Christine Jorgensen. . . . RT @meowmeowmia: the e in e-girl stands for evil @noobweps @pria_the_pluff @MsPatriciaGao I fell bad I couldn't reach out here but writing and talking over text is… https://t.co/Im1O2FatKu RT @huangdanlan: https://t.co/eR2WmalJSC @SithVenator @Xbox @bethesda @GearsofWar @OriTheGame @rage @ElderScrolls @PreyGame @dishonored first fanart I've seen @ValeriaHerreraJ Thank you! Spirit Awards 2021 Winners List: ‘Nomadland,’ ‘I May Destroy You,’ Riz Ahmed, Carey Mulligan Win Big https://t.co/mwCnyLPIwV via @indiewire RT @newsistaan: Protesters are on the move again here in Minneapolis, carrying a banner reading “The World is Watching” #Minneapolis #Chauv…
he/him UTD
 RT @MargBar

SyntaxError: invalid syntax (<ipython-input-15-1358657e8fee>, line 1)